In [9]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

from tensorflow import keras

In [6]:
train_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    image_size=(256, 256),
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='training')

val_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    image_size=(256, 256),
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='validation')

Found 25000 files belonging to 2 classes.
Using 17500 files for training.
Found 25000 files belonging to 2 classes.
Using 7500 files for validation.


In [7]:
model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(256, 256, 3)))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.SGD(),
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.Precision(),
        keras.metrics.Recall()
    ]
)

Andiamo ad inserire tre diversi callback:

1. un `ModelCheckpoint`, che salva i risultati intermedi nel file da noi specificati;
2. un `EarlyStopping`, che interrompe l'addestramento qualora i risultati ottenuti non migliorino;
3. un `TensorBoard`, che fa in modo che i risultati siano mostrati mediante TensorBoard.

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='checkpoints/checkpoints',
        save_weights_only=True,
        monitor='val_binary_accuracy',
        save_best_only=True),
    keras.callbacks.EarlyStopping(
        monitor='val_binary_accuracy',
        min_delta=0.1,
        patience=3,
        restore_best_weights=True),
    keras.callbacks.TensorBoard(log_dir='logs')
]

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
1094/1094 [==============================] - 166s 151ms/step - loss: 1830.7936 - binary_accuracy: 0.5005 - precision_1: 0.4982 - recall_1: 0.4331 - val_loss: 0.6957 - val_binary_accuracy: 0.5160 - val_precision_1: 0.5133 - val_recall_1: 0.7974
Epoch 2/10
1094/1094 [==============================] - 169s 154ms/step - loss: 0.6794 - binary_accuracy: 0.5359 - precision_1: 0.5303 - recall_1: 0.5944 - val_loss: 0.7476 - val_binary_accuracy: 0.5171 - val_precision_1: 0.5158 - val_recall_1: 0.7089
Epoch 3/10
1094/1094 [==============================] - 172s 157ms/step - loss: 0.6557 - binary_accuracy: 0.5662 - precision_1: 0.5491 - recall_1: 0.7198 - val_loss: 0.6940 - val_binary_accuracy: 0.4848 - val_precision_1: 0.4494 - val_recall_1: 0.0914
Epoch 4/10
1094/1094 [==============================] - 174s 159ms/step - loss: 0.6311 - binary_accuracy: 0.5882 - precision_1: 0.6020 - recall_1: 0.5107 - val_loss: 0.7033 - val_binary_accuracy: 0.4916 - val_precision_1: 0.4856 - val_recall